In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import random
from torchvision.utils import save_image
from torch.autograd import Variable
import os

In [2]:
class BIOF050_GAN:
    
    def __init__(self,data):
        self.data = data
        
    class Generator(nn.Module):
        def __init__(self,len_noise,hidden_dimensions,data_size):
            super(BIOF050_GAN.Generator, self).__init__()
            self.input_layer = nn.Linear(len_noise, hidden_dimensions)
            self.layer1 = nn.Linear(hidden_dimensions, hidden_dimensions)
            self.output_layer = nn.Linear(hidden_dimensions,data_size)
            self.batch_norm = nn.BatchNorm1d(hidden_dimensions)
            
            self.relu = nn.ReLU()
            self.tanh = nn.Tanh()
        
        def forward(self,batch):
            
            batch = self.input_layer(batch)
            batch = self.batch_norm(batch)
            batch = self.relu(batch)
            batch = self.layer1(batch)
            batch = self.batch_norm(batch)
            batch = self.relu(batch)
            batch = self.output_layer(batch)
            
            return self.tanh(batch)

    
    
    class Discriminator(nn.Module):
        def __init__(self,hidden_dimensions,data_size):
            
            super(BIOF050_GAN.Discriminator, self).__init__()
            
            self.input_layer = nn.Linear(data_size, hidden_dimensions)
            self.layer1 =  nn.Linear(hidden_dimensions, hidden_dimensions)
            self.output_layer = nn.Linear(hidden_dimensions, 2)
            
            self.batch_norm = nn.BatchNorm1d(hidden_dimensions)
            self.relu = nn.ReLU()
        
        def forward(self,batch):
        
            batch = self.input_layer(batch) 
            batch = self.batch_norm(batch)
            batch = self.relu(batch)
            batch = self.layer1(batch)
            batch = self.batch_norm(batch)
            batch = self.relu(batch)
            
            return nn.functional.softmax(self.output_layer(batch))
    
    def train_gan(self,n_epochs,noise_length,hidden_dimensions,batch_size,lr):

        train_batches = batchify(self.data,batch_size=batch_size)
        
        generator = BIOF050_GAN.Generator(noise_length,hidden_dimensions,len(self.data[0]))
        discriminator = BIOF050_GAN.Discriminator(hidden_dimensions,len(self.data[0]))
        
    
        loss = nn.BCELoss()
        disc_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
        gen_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

        for i in range(n_epochs):
            print(i)
            for ii in range(len(train_batches)):
                
                disc_optimizer.zero_grad()
                gen_optimizer.zero_grad()
                
                if i%2 == 0:
                    batch = train_batches[ii]
                    real_labels = torch.ones(len(batch))
                    fake_labels = torch.zeros(len(batch))
     

                    real_predictions = discriminator(torch.tensor(np.asarray(batch).astype(np.float32)))
                    disc_loss_r = loss(real_predictions[:,1], real_labels)
                    
                    noise = torch.randn(batch_size,noise_length)
                    fake_data = generator(noise)
                    test = discriminator(fake_data)
                    disc_loss_f = loss(test[:,1],fake_labels)

                    disc_loss = disc_loss_r + disc_loss_f
                    
                    
                    disc_loss.backward()
                    disc_optimizer.step()
                
                else:
                    noise = torch.randn(batch_size,noise_length)
                    fake_data = generator(noise)
                    
                    test = discriminator(fake_data)
                
                    gen_loss = loss(test[:,1],real_labels)
                    if ii%500 == 0:
                        print(gen_loss)
                    gen_loss.backward()
                    gen_optimizer.step()
                    
            fake_data = (fake_data + 1)/ 2
            fake_data = fake_data.clamp(0, 1)
            fake_data = fake_data.view(fake_data.size(0), 1, 28, 28)
            save_image(fake_data.data,os.path.join(os.getcwd(), 'fake_images' + str(i) + '.png'))
            
            

def batchify(data,batch_size=16):
    
    batches= []
                                         
    for n in range(0,len(data),batch_size):
        if n+batch_size < len(data):
            batches.append(data[n:n+batch_size])

    if len(data)%batch_size > 0:
        batches.append(data[len(data)-(len(data)%batch_size):len(data)])
      
    return batches


In [3]:
data = torchvision.datasets.MNIST(
    root = './data/MNIST',
    download = True)

data = data.data

newdata = []

for image in data:
    image = np.ravel(image).astype(np.float64)
    image *= 1/image.max()
    newdata.append(image)

In [4]:
testclass = BIOF050_GAN(newdata)
model = testclass.train_gan(n_epochs=20,noise_length=50,hidden_dimensions=300,batch_size=16,lr=0.001)

0


<ipython-input-2-1da28a0469a2>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(self.output_layer(batch))


1
tensor(8.5828, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0020, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0018, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0032, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0006, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0003, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0010, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0008, grad_fn=<BinaryCrossEntropyBackward>)
2
3
tensor(8.8786, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0004, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0007, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0001, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0003, grad_fn=<BinaryCrossEntropyBackward>)


KeyboardInterrupt: 